<a href="https://colab.research.google.com/github/Emmanuel2000x/DataScience-Trabajar-con-funciones/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

In [22]:
#Se importan las librerias necesarias para el proyecto
import pandas as pd
import numpy as np

In [23]:
#Se le la URL del .json con la informacion que otorga la API
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"
datos_json = pd.read_json(url)

In [24]:
#El DF original cuenta 7267 registros con 6 columnas |customerID|Churn|customer|phone|internet |account|
print(datos_json.shape)
print(datos_json.head(3))

(7267, 6)
   customerID Churn                                           customer  \
0  0002-ORFBO    No  {'gender': 'Female', 'SeniorCitizen': 0, 'Part...   
1  0003-MKNFE    No  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
2  0004-TLHLJ   Yes  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   

                                             phone  \
0   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
1  {'PhoneService': 'Yes', 'MultipleLines': 'Yes'}   
2   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   

                                            internet  \
0  {'InternetService': 'DSL', 'OnlineSecurity': '...   
1  {'InternetService': 'DSL', 'OnlineSecurity': '...   
2  {'InternetService': 'Fiber optic', 'OnlineSecu...   

                                             account  
0  {'Contract': 'One year', 'PaperlessBilling': '...  
1  {'Contract': 'Month-to-month', 'PaperlessBilli...  
2  {'Contract': 'Month-to-month', 'PaperlessBilli...  


In [25]:
#Creamos una funcion encargada de normalizar todas las columnas anidadas
def normalize_nested_df(df: pd.DataFrame) -> pd.DataFrame:
  """
  Normaliza de forma recursiva las columnas anidadas de un DF
  Args:
    df (pd.DataFrame): Input DataFrame with potential nested structures.
  Returns:
    pd.DataFrame: Fully normalized (flattened) DataFrame.
  """
  df = df.copy()
  new_columns = []
  for col in df.columns:

    if df[col].apply(lambda x: isinstance(x, dict)).any():
      # Expand dictionaries into separate columns
      expanded = pd.json_normalize(df[col]).add_prefix(f'{col}_')
      df = df.drop(columns=[col])
      df = pd.concat([df, expanded], axis=1)
      return normalize_nested_df(df)  # Recursively normalize
    else:
      new_columns.append(col)

  return df


In [26]:
#Normalizacion del DF utilizando la funcion creada
flatt_DF = normalize_nested_df(datos_json)
#Ahora tenemos 21 columnas y ninguna con info anidada
print(flatt_DF.shape)
print(flatt_DF.head(3))

(7267, 21)
   customerID Churn customer_gender  customer_SeniorCitizen customer_Partner  \
0  0002-ORFBO    No          Female                       0              Yes   
1  0003-MKNFE    No            Male                       0               No   
2  0004-TLHLJ   Yes            Male                       0               No   

  customer_Dependents  customer_tenure phone_PhoneService phone_MultipleLines  \
0                 Yes                9                Yes                  No   
1                  No                9                Yes                 Yes   
2                  No                4                Yes                  No   

  internet_InternetService  ... internet_OnlineBackup  \
0                      DSL  ...                   Yes   
1                      DSL  ...                    No   
2              Fiber optic  ...                    No   

  internet_DeviceProtection internet_TechSupport internet_StreamingTV  \
0                        No              

In [27]:
#Lista que almacena las Variables/Columnas que contiene el DF
Datos_dic = list(flatt_DF.columns)
print(Datos_dic)

['customerID', 'Churn', 'customer_gender', 'customer_SeniorCitizen', 'customer_Partner', 'customer_Dependents', 'customer_tenure', 'phone_PhoneService', 'phone_MultipleLines', 'internet_InternetService', 'internet_OnlineSecurity', 'internet_OnlineBackup', 'internet_DeviceProtection', 'internet_TechSupport', 'internet_StreamingTV', 'internet_StreamingMovies', 'account_Contract', 'account_PaperlessBilling', 'account_PaymentMethod', 'account_Charges.Monthly', 'account_Charges.Total']


In [28]:
#Clasificamos el tipo de dato de cada columna
Col_Bol = ['Churn', 'customer_SeniorCitizen','customer_Partner',
        'customer_Dependents','phone_PhoneService', 'phone_MultipleLines',
        'internet_OnlineSecurity', 'internet_OnlineBackup','internet_DeviceProtection', 'internet_TechSupport',
        'internet_StreamingTV', 'internet_StreamingMovies','account_PaperlessBilling']
Col_Str = ['customerID','customer_gender', 'internet_InternetService',
           'account_Contract','account_PaymentMethod']
Col_Flo = ['account_Charges.Monthly', 'account_Charges.Total']
Col_Int = ['customer_tenure']

#🔧 Transformación

## Conocemos el tipo de dato con el que viene cada columna


In [29]:
#Exploramos las columnas del data set y verificamos los tipos de datos
flatt_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer_gender            7267 non-null   object 
 3   customer_SeniorCitizen     7267 non-null   int64  
 4   customer_Partner           7267 non-null   object 
 5   customer_Dependents        7267 non-null   object 
 6   customer_tenure            7267 non-null   int64  
 7   phone_PhoneService         7267 non-null   object 
 8   phone_MultipleLines        7267 non-null   object 
 9   internet_InternetService   7267 non-null   object 
 10  internet_OnlineSecurity    7267 non-null   object 
 11  internet_OnlineBackup      7267 non-null   object 
 12  internet_DeviceProtection  7267 non-null   object 
 13  internet_TechSupport       7267 non-null   objec

In [30]:
flatt_DF.dtypes

,0
customerID,object
Churn,object
customer_gender,object
customer_SeniorCitizen,int64
customer_Partner,object
customer_Dependents,object
customer_tenure,int64
phone_PhoneService,object
phone_MultipleLines,object
internet_InternetService,object


In [31]:
#De origen el data set parece no contener errores nulos
flatt_DF.isnull().sum()

,0
customerID,0
Churn,0
customer_gender,0
customer_SeniorCitizen,0
customer_Partner,0
customer_Dependents,0
customer_tenure,0
phone_PhoneService,0
phone_MultipleLines,0
internet_InternetService,0


In [32]:
flatt_DF.iloc[975]

,975
customerID,1371-DWPAZ
Churn,No
customer_gender,Female
customer_SeniorCitizen,0
customer_Partner,Yes
customer_Dependents,Yes
customer_tenure,0
phone_PhoneService,No
phone_MultipleLines,No phone service
internet_InternetService,DSL


In [33]:
#Realizamos la modificacion de los datos vacios por NaN
mod_DF = flatt_DF.replace(['', ' '], np.nan)

In [42]:
#Una vez transformados los datos detectamos lo siguiente
mod_DF.isnull().sum()

,0
customerID,0
Churn,224
customer_gender,0
customer_SeniorCitizen,0
customer_Partner,0
customer_Dependents,0
customer_tenure,0
phone_PhoneService,0
phone_MultipleLines,0
internet_InternetService,0


In [43]:
flatt_DF.iloc[errores_list]

,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,...,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges.Monthly,account_Charges.Total
975,1371-DWPAZ,True,Female,False,True,True,0,True,True,DSL,...,True,True,True,True,True,Two year,True,Credit card (automatic),56.05,NaN
1775,2520-SGTTA,True,Female,False,True,True,0,True,True,No,...,True,True,True,True,True,Two year,True,Mailed check,20.00,NaN
1955,2775-SEFEE,True,Male,False,True,True,0,True,True,DSL,...,True,True,True,True,True,Two year,True,Bank transfer (automatic),61.90,NaN
2075,2923-ARZLG,True,Male,False,True,True,0,True,True,No,...,True,True,True,True,True,One year,True,Mailed check,19.70,NaN
2232,3115-CZMZD,True,Male,False,True,True,0,True,True,No,...,True,True,True,True,True,Two year,True,Mailed check,20.25,NaN
2308,3213-VVOLG,True,Male,False,True,True,0,True,True,No,...,True,True,True,True,True,Two year,True,Mailed check,25.35,NaN
2930,4075-WKNIU,True,Female,False,True,True,0,True,True,DSL,...,True,True,True,True,True,Two year,True,Mailed check,73.35,NaN
3134,4367-NUYAO,True,Male,False,True,True,0,True,True,No,...,True,True,True,True,True,Two year,True,Mailed check,25.75,NaN
3203,4472-LVYGI,True,Female,False,True,True,0,True,True,DSL,...,True,True,True,True,True,Two year,True,Bank transfer (automatic),52.55,NaN
4169,5709-LVOEQ,True,Female,False,True,True,0,True,True,DSL,...,True,True,True,True,True,Two year,True,Mailed check,80.85,NaN


In [36]:
for col in Col_Flo:
    print(f"Unique values in {col}:")
    # Convert to string type temporarily to handle mixed types if necessary
    print(flatt_DF[col].astype(str).unique())

Unique values in account_Charges.Monthly:
['65.6' '59.9' '73.9' ... '91.75' '68.8' '67.85']
Unique values in account_Charges.Total:
['593.3' '542.4' '280.85' ... '742.9' '4627.65' '3707.6']


In [37]:
#Eliminando los espacios del la columna cargos totales
flatt_DF[Col_Flo] = flatt_DF[Col_Flo].replace(' ', np.nan)
#convirtiendo el tipo de dato
flatt_DF[Col_Flo] = flatt_DF[Col_Flo].apply(pd.to_numeric, errors='coerce')

In [38]:
flatt_DF[Col_Bol] = flatt_DF[Col_Bol].apply(lambda x: x.replace('No','0').replace('Yes','1'))
flatt_DF[Col_Bol] = flatt_DF[Col_Bol].astype('bool')

In [44]:
flatt_DF[Col_Str] = flatt_DF[Col_Str].astype('string')

flatt_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   string 
 1   Churn                      7267 non-null   bool   
 2   customer_gender            7267 non-null   string 
 3   customer_SeniorCitizen     7267 non-null   bool   
 4   customer_Partner           7267 non-null   bool   
 5   customer_Dependents        7267 non-null   bool   
 6   customer_tenure            7267 non-null   int64  
 7   phone_PhoneService         7267 non-null   bool   
 8   phone_MultipleLines        7267 non-null   bool   
 9   internet_InternetService   7267 non-null   string 
 10  internet_OnlineSecurity    7267 non-null   bool   
 11  internet_OnlineBackup      7267 non-null   bool   
 12  internet_DeviceProtection  7267 non-null   bool   
 13  internet_TechSupport       7267 non-null   bool 

#📊 Carga y análisis

In [40]:
flatt_DF.sample(5)

,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,...,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges.Monthly,account_Charges.Total
2392,3339-EAQNV,True,Male,True,True,True,72,True,True,Fiber optic,...,True,True,True,True,True,One year,True,Credit card (automatic),97.25,7133.1
4945,6749-UTDVX,True,Male,False,True,True,72,True,True,Fiber optic,...,True,True,True,True,True,Two year,True,Electronic check,103.65,7634.8
141,0230-UBYPQ,True,Male,True,True,True,63,True,True,DSL,...,True,True,True,True,True,One year,True,Bank transfer (automatic),36.10,2298.9
3966,5423-BHIXO,True,Female,False,True,True,32,True,True,DSL,...,True,True,True,True,True,Month-to-month,True,Electronic check,54.20,1739.6
1690,2393-DIVAI,True,Female,False,True,True,3,True,True,No,...,True,True,True,True,True,Month-to-month,True,Mailed check,20.00,61.7


In [41]:
flatt_DF['account_Charges.Total


SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-41-6db6f95f5759>, line 1)

#📄Informe final